<a href="https://colab.research.google.com/github/SwordForShinobi/Retail-recommender-system/blob/main/RecSystem_Naive_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import random

In [ ]:
class Recommender:
  '''data must be DataFrame, others args(columns' names) - strings'''

  def __init__(self, data, order_column, user_column, product_column):# класс будет сам группировать исходный датафрейм для дальнейшей работы
      self.data = data
      self.order_column = order_column
      self.user_column = user_column
      self.product_column = product_column
      self.grouped = self.data.groupby([self.order_column, self.user_column])[self.product_column].apply(lambda x: list(set(x))).reset_index()

  def show_gpouped(self): # просто для просмотра группировки
    df = self.grouped
    return df
  
  def predict_user(self, user_number):
    '''input integer'''
    if len(self.grouped[self.grouped[self.user_column] == user_number][self.user_column]) > 0:
      example = self.grouped[self.grouped[self.user_column] == user_number]
      items = example.product_id.explode().value_counts().reset_index()
      items.columns = ['product_id', 'amount']
      print(f'For user# {user_number} we suggest next items: {items['product_id'].tolist()}')
    else:
      print('There is no such user =(')
    

  def predict_many_users(self, users_ids):
    '''users_ids - list of integers'''
    who = self.grouped.loc[self.grouped[self.user_column].isin(users_ids)]
    users_row = list(set(who[self.user_column]))
    top10_row = []

    for i in users_ids:
      example = self.grouped[self.grouped[self.user_column] == i]
      items = example.product_id.explode().value_counts().reset_index()
      items.columns = ['product_id', 'amount']
      top10 = items.head(10).product_id.tolist()
      top10_row.append(top10)

    answer = pd.DataFrame(users_row, columns=['user'])
    answer['top10'] = top10_row
    return answer

  def lol_prediction(self, users_ids, top_what=25):
    # Это гениальный метод, он формирует топ n товаров, (n получает на вход), и рандомом рекоммендует оттуда топ10. Сильно? =)))
    # Такой подход уже можно назвать рекоммендательной системой? Мои философские воззрения говорят, что можно. 
    '''users_ids - list of integers
        top_what - set of each user preferences, how many top prefs we take'''
    who = self.grouped.loc[self.grouped[self.user_column].isin(users_ids)]
    users_row = list(set(who[self.user_column]))
    top10_row = []

    for i in users_row:
      example = self.grouped[self.grouped[self.user_column] == i]
      items = example.product_id.explode().value_counts().reset_index()
      items.columns = ['product_id', 'amount']
      top_smth = items.head(top_what).product_id.tolist()
      top10 = random.choices(top_smth, k=10)
      top10_row.append(top10)

    answer = pd.DataFrame(users_row, columns=['user'])
    answer['top10'] = top10_row
    return answer

In [ ]:
# Reading transactions dataframe
rec = Recommender(trans, 'order_id', 'user_id', 'product_id')

In [ ]:
# Showing grouped dataframe (orders counted for each user)
rec.show_gpouped().head()

In [ ]:
# Predictions for a user
rec.predict_user(1)

In [ ]:
# Predictions for desired users
rec.lol_prediction(trans['user_id'].unique().tolist(), 10)